In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import mobilenet_v2
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model, Sequential
from keras import models
from keras import layers
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from util.plots import plot_history
from util.data_generator import make_data_generator
from keras.callbacks import ModelCheckpoint

from keras import backend as K

Using TensorFlow backend.


In [2]:
cfg = K.tf.ConfigProto() # Config to dynamically add memory
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))# set this TensorFlow session as the default session for Keras.

In [3]:
batch_size = 32
train_path = '../dataset3/res_still/train'
test_path = '../dataset3/res_still/test'
train_generator, test_generator = make_data_generator(train_path, test_path, load_ram=False)

Found 1875 images belonging to 8 classes.
Found 472 images belonging to 8 classes.


In [4]:
base_model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top = False, input_shape=(224, 224, 3))

In [5]:
for layer in base_model.layers:
    layer.trainable=False
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

base_model.summary()
model.summary()
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [6]:
# Checkpoint to save best model
#file_path = 'saved_models/mobiletnetweight.hdf5'
#checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callback_list = [checkpoint]
callback_list = []

In [7]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch=1875 // batch_size,
                    epochs=50,
                    validation_data=test_generator,
                    validation_steps=472 // batch_size,
                    use_multiprocessing=True,
                    workers=2,
                    callbacks=callback_list)

Epoch 1/50
58/58 [==============================] - 37s 641ms/step - loss: 2.1503 - acc: 0.1775 - val_loss: 2.0253 - val_acc: 0.1585
Epoch 2/50
58/58 [==============================] - 39s 665ms/step - loss: 1.8813 - acc: 0.2720 - val_loss: 1.8798 - val_acc: 0.2659
Epoch 3/50
58/58 [==============================] - 39s 676ms/step - loss: 1.7528 - acc: 0.3311 - val_loss: 1.7690 - val_acc: 0.3318
Epoch 4/50
58/58 [==============================] - 31s 526ms/step - loss: 1.5669 - acc: 0.4058 - val_loss: 1.6864 - val_acc: 0.3364
Epoch 5/50
58/58 [==============================] - 32s 544ms/step - loss: 1.4665 - acc: 0.4457 - val_loss: 1.5382 - val_acc: 0.4091
Epoch 6/50
58/58 [==============================] - 29s 504ms/step - loss: 1.3815 - acc: 0.4857 - val_loss: 1.6028 - val_acc: 0.3568
Epoch 7/50
58/58 [==============================] - 32s 548ms/step - loss: 1.2493 - acc: 0.5425 - val_loss: 1.5294 - val_acc: 0.3932
Epoch 8/50
57/58 [============================>.] - ETA: 0s - loss: 1

Process ForkPoolWorker-29:
Process ForkPoolWorker-27:
Traceback (most recent call last):
Process ForkPoolWorker-28:
Process ForkPoolWorker-30:
  File "/home/emil/anaconda3/envs/NN1.8/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/emil/anaconda3/envs/NN1.8/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/emil/anaconda3/envs/NN1.8/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/emil/anaconda3/envs/NN1.8/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/emil/anaconda3/envs/NN1.8/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/emil/anaconda3/envs/NN1.8/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/

KeyboardInterrupt: 

In [ ]:
plot_history(history, save=False, path='nets/mobile_net/', name='plot4')

In [ ]:
#from importlib import reload
#import util.plots
#reload(util.plots)
#from util.plots import plot_history
#plot_history(history, mean_N=3, save=True, path='nets/mobile_net/', name='plot4', ylim_acc=(0.4,1.000), ylim_loss=(0.0,1.6))